# Other
> Fix common OpenCV python DLL load failed error when building python bindings from source

- toc: true 
- badges: true
- comments: true
- sticky_rank: 1
- author: cudawarped
- image: images/diagram.png
- categories: [opencv, cuda, python]

So your on Windows using python >= 3.8, you have built OpenCV >= 4.6 from source, manually copied .. python path them to your distributions site... directory, added the path to the directory containing opencv.dll and any other shared library dependencies and your seeing something similar to the following.

In [1]:
#import os
#os.environ["OpenCV_LOADER_DEBUG"]="1"
import cv2

ModuleNotFoundError: No module named 'cv2'

What's going on? Although the message is quite explicit regarding the cause, it doesn't really help with finding a solution. In a nutshell python has found .pyd (shared lib), tried and failed to load it because a it can't find a dependant shared library. Now the advice I have seen online regarding this is to dig out trusty dependancy walker, load the .pyd and see which dependencies the system can't find.

Now this is solid advice if we had a C++ application and/or we were using python < 3.8 (which uses the system/user path for dll re), however we are not, so even if dependency walker can't detect any problems we may still be facing the above error.

There is a missing dll, which needs to be added with ... before calling import cv2, now if you have installed the python loader with OpenCV then you will likeliy need to search with .... to find it, however if you have not you may be able to try the quick fix below may work for you


### Quick fix when not using the OpenCV python loader

Using the flag OPENCV_SKIP_PYTHON_LOADER=ON prevents the python bindings from being installed when you build install OpenCV, therefore depending on how you installed cv2 there shouldn't be a cv2 directory inside lib/site-sitepackages and you should something similar to the following

There are multiple reasons that the OpenCV python loader may be missing
1) You didn't build install

2) OPENCV_SKIP_PYTHON_LOADER=ON was passed to cmake

3) The lib/site-packages/cv2 diretory was deleted

Whatever the reason if you are seeing the above error you have likely manually "installed" the python bindings either with

1) set PYTHONPATH="OPENCV_BUILD_PATH/lib/python3/"

2) or by manually copying .... pyd to the lib/site-packages/

Whichever way this was performed, you will be missing (unless your using a distribution which automatically adds .. directories from the path to the dll search path)

os.add_dll_directory("OPENCV_BUID_PATH/bin/");

in this case there will be at least one dll search path which needs to be added.

In [24]:
#collapse-output
import os;
OPENCV_BUID_PATH = "D:/build/opencv/cuda_11_7_cc_8_6_ff_sym/"
os.add_dll_directory(OPENCV_BUID_PATH + "/bin/");
import cv2

ImportError: DLL load failed while importing cv2: The specified module could not be found.

If that didn't work and you are using the CUDA modules you will also need to add 
os.add_dll_directory("C:\\Program Files\\NVIDIA GPU Computing Toolkit\\CUDA\\v11.7\\bin");, if that doesn't work then you need to locate the missing dll and manually add it, seee ...

In [25]:
#collapse-output
os.add_dll_directory("C:\\Program Files\\NVIDIA GPU Computing Toolkit\\CUDA\\v11.7\\bin");
import cv2

ImportError: DLL load failed while importing cv2: The specified module could not be found.

In [13]:
import sysconfig
import os
os.listdir(sysconfig.get_paths()["purelib"]+"/cv2")

FileNotFoundError: [WinError 3] The system cannot find the path specified: 'C:\\Users\\b\\mambaforge\\Lib\\site-packages/cv2'

In this case you will definitely be missing the directory containing .... and ... and if you built with CUDA you will also be missing ... therefore before doing anything else 

## You have installed the python loader